In [1]:
!pip install easyocr       
!pip install bs4
!pip install Pattern
!pip3 install torch torchvision torchaudio
!pip install "opencv-python-headless<4.3"

     |████████████████████████████████| 63.6 MB 47 kB/s 
     |████████████████████████████████| 47.7 MB 1.5 MB/s 
     |████████████████████████████████| 22.2 MB 2.1 MB/s 
     |████████████████████████████████| 87 kB 6.7 MB/s 
     |████████████████████████████████| 81 kB 8.9 MB/s 
     |████████████████████████████████| 5.6 MB 34.7 MB/s 
     |████████████████████████████████| 5.6 MB 31.4 MB/s 
     |████████████████████████████████| 419 kB 33.6 MB/s 
     |████████████████████████████████| 104 kB 45.4 MB/s 
     |████████████████████████████████| 3.6 MB 35.2 MB/s 
  Created wheel for Pattern: filename=Pattern-3.6-py3-none-any.whl size=22332721 sha256=3db76701a94576ff988b9c438a3422dc52957517021ed1b04102447a6b3aab49
  Stored in directory: /root/.cache/pip/wheels/8d/1f/4e/9b67afd2430d55dee90bd57618dd7d899f1323e5852c465682
  Created wheel for mysqlclient: filename=mysqlclient-2.1.0-cp37-cp37m-linux_x86_64.whl size=99972 sha256=8641602eead656b8cae0e620d890576038cc70803e4c2c2892d004d0a49

In [16]:
import json

import easyocr

from pattern.web import Crawler, DEPTH
from pattern.web import Wikipedia
from pattern.web import Google, SEARCH, plaintext

import requests
from bs4 import BeautifulSoup , SoupStrainer
import re

# Task 1

In [3]:
reader = easyocr.Reader(["en"])

In [4]:
result = reader.readtext("physics.png")

In [5]:
result

[([[6, 4], [1694, 4], [1694, 35], [6, 35]],
  'Physics is the natural science that studies matter,[al its fundamental constituents, its motion and behavior through space and time, and the related entities of energy and force [2]',
  0.7029828707932679),
 ([[4, 34], [1246, 34], [1246, 64], [4, 64]],
  'Physics is one of the most fundamental scientific disciplines, and its main goal is to understand how the universe behaves [b][3I[4][5]',
  0.5677980052134863),
 ([[4, 74], [1710, 74], [1710, 102], [4, 102]],
  'Physics is one of the oldest academic disciplines and, through its inclusion of astronomy, perhaps the oldest [6] Over much of the past two millennia, physics, chemistry, biology, and',
  0.6872995065998662),
 ([[4, 104], [1654, 104], [1654, 133], [4, 133]],
  'certain branches of mathematics were a part of natural philosophy, but during the Scientific Revolution in the 17th century these natural sciences emerged as unique research',
  0.7817703856363454),
 ([[4, 132], [1748, 132]

In [6]:
result_plain = reader.readtext("physics.png", detail=0)

In [7]:
result_plain

['Physics is the natural science that studies matter,[al its fundamental constituents, its motion and behavior through space and time, and the related entities of energy and force [2]',
 'Physics is one of the most fundamental scientific disciplines, and its main goal is to understand how the universe behaves [b][3I[4][5]',
 'Physics is one of the oldest academic disciplines and, through its inclusion of astronomy, perhaps the oldest [6] Over much of the past two millennia, physics, chemistry, biology, and',
 'certain branches of mathematics were a part of natural philosophy, but during the Scientific Revolution in the 17th century these natural sciences emerged as unique research',
 'endeavors in their own right Ic] Physics intersects with many interdisciplinary areas of research, such as biophysics and quantum chemistry, and the boundaries of physics are not rigidly',
 'defined. New ideas in physics often explain the fundamental mechanisms studied by other sciences[3] and suggest new

In [8]:
" ".join(result_plain)

'Physics is the natural science that studies matter,[al its fundamental constituents, its motion and behavior through space and time, and the related entities of energy and force [2] Physics is one of the most fundamental scientific disciplines, and its main goal is to understand how the universe behaves [b][3I[4][5] Physics is one of the oldest academic disciplines and, through its inclusion of astronomy, perhaps the oldest [6] Over much of the past two millennia, physics, chemistry, biology, and certain branches of mathematics were a part of natural philosophy, but during the Scientific Revolution in the 17th century these natural sciences emerged as unique research endeavors in their own right Ic] Physics intersects with many interdisciplinary areas of research, such as biophysics and quantum chemistry, and the boundaries of physics are not rigidly defined. New ideas in physics often explain the fundamental mechanisms studied by other sciences[3] and suggest new avenues of research 

# Task 2

In [9]:
class Crawl(Crawler): 
    def visit(self, link, source=None):
        st = "VISITED: " + str(link.url) + " | FROM: " + str(link.referrer)
        print(st)
    def fail(self, link):
        print("failed:", repr(link.url))
        
url0 = "https://www.nature.com/"
p = Crawl(links=[url0], delay=0.2)
num_visited = 0
while (not p.done and num_visited < 10) :
    p.crawl(
        method=DEPTH, 
        cached=False, 
        throttle=1
    )
    num_visited += 1

VISITED: https://www.nature.com/ | FROM: 
VISITED: https://www.nature.com/#content | FROM: https://www.nature.com/
VISITED: https://www.nature.com/siteindex | FROM: https://www.nature.com/
VISITED: https://www.nature.com/#search-menu | FROM: https://www.nature.com/
VISITED: https://www.nature.com/nams/svc/myaccount | FROM: https://www.nature.com/
VISITED: https://www.nature.com/#explore | FROM: https://www.nature.com/
VISITED: https://www.nature.com/#about-the-journal | FROM: https://www.nature.com/
VISITED: https://www.nature.com/#publish-with-us | FROM: https://www.nature.com/
VISITED: https://www.nature.com/nature/subscribe | FROM: https://www.nature.com/
VISITED: https://www.nature.com/my-account/alerts/subscribe-journal?list-id=1 | FROM: https://www.nature.com/


In [10]:
p.visited

{'https://www.nature.com/': True,
 'https://www.nature.com/#about-the-journal': True,
 'https://www.nature.com/#content': True,
 'https://www.nature.com/#explore': True,
 'https://www.nature.com/#publish-with-us': True,
 'https://www.nature.com/#search-menu': True,
 'https://www.nature.com/my-account/alerts/subscribe-journal?list-id=1': True,
 'https://www.nature.com/nams/svc/myaccount': True,
 'https://www.nature.com/nature/subscribe': True,
 'https://www.nature.com/siteindex': True}

# Task 3

In [11]:
site = "https://www.nature.com/"

response = requests.get(site)

soup = BeautifulSoup(response.text, "html.parser")
img_tags = soup.find_all("img")

img_urls = ["https:" + img["src"] for img in img_tags]

In [12]:
img_urls

['https://pubads.g.doubleclick.net/gampad/ad?iu=/285/nature.com/homepage&sz=728x90&pos=top;type=homepage;path=/nature',
 'https://media.springernature.com/full/nature-cms/uploads/product/nature/header-86f1267ea01eccd46b530284be10585e.svg',
 'https://media.springernature.com/w735h400/nature-cms/uploads/cms/pages/2913/top_item_image/2022-02-10_NEWS_Proxima-centauri-planet_THUMB-26d3637113413a6a186a93289d5e986c.jpg',
 'https://media.springernature.com/w290h158/magazine-assets/d41586-022-00376-0/d41586-022-00376-0_20112588.jpg',
 'https://media.springernature.com/w290h158/magazine-assets/d41586-022-00391-1/d41586-022-00391-1_20117134.jpg',
 'https://media.springernature.com/w290h158/springer-static/image/art%3A10.1038%2Fs41586-021-04357-7/MediaObjects/41586_2021_4357_Fig1_HTML.png',
 'https://media.springernature.com/w290h158/magazine-assets/d41586-022-00396-w/d41586-022-00396-w_17300602.png',
 'https://media.springernature.com/w440/springer-static/cover-hires/journal/41586/602/7896',
 'ht

In [26]:
for txt in soup.find_all("p"):
  print(txt.get_text())

Thank you for visiting nature.com. You are using a browser version with limited support for CSS. To obtain
            the best experience, we recommend you use a more up to date browser (or turn off compatibility mode in
            Internet Explorer). In the meantime, to ensure continued support, we are displaying the site without styles
            and JavaScript.
Advertisement
Shifts in starlight from Proxima Centauri reveal its third planet — a world orbiting within the star’s habitable zone.

Researchers take stock of the US$1.8-billion initiative’s first five years as Biden announces ambitious target.
The experimental Joint European Torus has doubled the record for the amount of energy made from fusing atoms — the process that powers the Sun. 
Using the game Gran Turismo, an agent was trained with a combination of deep reinforcement learning algorithms and specialized training scenarios, demonstrating success against championship-level human racers.
Three people once paralysed b

In [29]:
import requests, bs4
r = requests.get('https://www.nature.com/')
soup = bs4.BeautifulSoup(r.text, 'lxml')

p_tags = soup.find_all('p')
for p in p_tags:
    if p.string:
        print(p.string)

Thank you for visiting nature.com. You are using a browser version with limited support for CSS. To obtain
            the best experience, we recommend you use a more up to date browser (or turn off compatibility mode in
            Internet Explorer). In the meantime, to ensure continued support, we are displaying the site without styles
            and JavaScript.
Advertisement
Shifts in starlight from Proxima Centauri reveal its third planet — a world orbiting within the star’s habitable zone.

Researchers take stock of the US$1.8-billion initiative’s first five years as Biden announces ambitious target.
The experimental Joint European Torus has doubled the record for the amount of energy made from fusing atoms — the process that powers the Sun. 
Using the game Gran Turismo, an agent was trained with a combination of deep reinforcement learning algorithms and specialized training scenarios, demonstrating success against championship-level human racers.
Three people once paralysed b

In [27]:
title = soup.title.text

In [28]:
title

'Nature'

In [17]:
links= [
        link.get("href")
        for link in soup.find_all("a",attrs={"href":re.compile("http://")})  
]

In [18]:
links

['http://feeds.nature.com/nature/rss/current',
 'http://dx.doi.org/10.1038/d41586-022-00293-2',
 'http://dx.doi.org/10.1038/d41586-022-00312-2',
 'http://dx.doi.org/10.1038/d41586-022-00317-x',
 'http://dx.doi.org/10.1038/s41586-021-04357-7',
 'http://www.nature.com/natureevents/science',
 'http://www.nature.com/naturejobs/science',
 'http://feeds.nature.com/nature/rss/current',
 'http://www.naturechina.com']